In [1]:
import pandas as pd
data = pd.read_csv('breast-cancer-wisconsin.csv', encoding = 'utf-8')
data.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
X = data[data.columns[1:10]]
y = data[['Class']]

In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [9]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

C:\Users\choim\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.984375

In [10]:
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[329   4]
 [  4 175]]


In [11]:
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print("분류예측 리포트 : \n", cfreport_train)

분류예측 리포트 : 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       333
           1       0.98      0.98      0.98       179

    accuracy                           0.98       512
   macro avg       0.98      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512



In [12]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9649122807017544

In [13]:
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트 데이터 오차행렬 : \n", confusion_test)

테스트 데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


In [15]:
from sklearn.metrics import classification_report
cfreport_test = classification_report(y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.99      0.95      0.97       111
           1       0.92      0.98      0.95        60

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171



그리드 서치

In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
param_grid = [{'kernel' : ['rbf'], 'C' : [0.001, 0.01, 0.1, 1, 10, 100], 'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]},
             {'kernel' : ['linear'], 'C' : [0.001, 0.01, 0.1, 1, 10, 100], 'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]}]
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv = 5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['rbf']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['linear']}])

In [21]:
print("best parameter : {}".format(grid_search.best_params_))
print("best score : {:.4f}".format(grid_search.best_score_))
print("test set score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

best parameter : {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
best score : 0.9746
test set score : 0.9591


랜덤 서치

In [22]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {'kernel' : ['rbf'], 'C' : randint(low = 0.001, high = 100), 'gamma' : randint(low = 0.001, high = 100)}
random_search = RandomizedSearchCV(SVC(), param_distribs, n_iter = 100, cv = 5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E9E9BE6AC0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E9E9A40EB0>,
                                        'kernel': ['rbf']})

In [23]:
print("best paramer : {}".format(random_search.best_params_))
print("best score : {:.4f}".format(random_search.best_score_))
print("test set score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

best paramer : {'C': 18, 'gamma': 7, 'kernel': 'rbf'}
best score : 0.9609
test set score : 0.9649


**회귀**

In [24]:
data2 = pd.read_csv('house_price.csv', encoding = 'utf-8')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [25]:
x = data2[data2.columns[0:5]]
y = data2[['house_value']]

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)

In [27]:
scaler.fit(x_train)
x_scaled_train = scaler.transform(x_train)
x_scaled_test = scaler.transform(x_test)

In [28]:
from sklearn.svm import SVR
model = SVR(kernel = 'poly')
model.fit(x_scaled_train, y_train)
pred_train = model.predict(x_scaled_train)
model.score(x_scaled_train, y_train)

0.4411532001585845

In [30]:
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test)

0.4569848508565627

In [31]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련 데이터 RMSE : ", np.sqrt(MSE_train))
print("평가 데이터 RMSE : ", np.sqrt(MSE_test))

훈련 데이터 RMSE :  71350.56690936949
평가 데이터 RMSE :  70447.96409927841


그리드 서치

In [32]:
param_grid = {'kernel' : ['poly'], 'C' : [0.01, 0.1, 1, 10], 'gamma' : [0.01, 0.1, 1, 10]}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(SVR(kernel = 'poly'), param_grid, cv = 5)
grid_search.fit(x_scaled_train, y_train)

GridSearchCV(cv=5, estimator=SVR(kernel='poly'),
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['poly']})

In [33]:
print("best parameter : {}".format(grid_search.best_params_))
print("best score : {:.4f}".format(grid_search.best_score_))
print("test set score : {:.4f}".format(grid_search.score(x_scaled_test, y_test)))

best parameter : {'C': 10, 'gamma': 10, 'kernel': 'poly'}
best score : 0.5548
test set score : 0.5716


**랜덤 서치**

In [35]:

param_distribs = {'kernel' : ['poly'], 'C' : randint(low = 0.01, high = 10), 'gamma' : randint(low = 0.01, high = 10)}
random_search = RandomizedSearchCV(SVR(kernel = 'poly'), param_distribs, n_iter = 20, cv = 5)
random_search.fit(x_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVR(kernel='poly'), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E9E9BAE700>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E9E65E73A0>,
                                        'kernel': ['poly']})

In [36]:
print("best parameter : {}".format(random_search.best_params_))
print("best score : {:.4f}".format(random_search.best_score_))
print("test set score : {:.4f}".format(random_search.score(x_scaled_test, y_test)))

best parameter : {'C': 7, 'gamma': 9, 'kernel': 'poly'}
best score : 0.5420
test set score : 0.5610
